In [ ]:
# can you edit this tom?

#i have been able to edit from VScode

In [ ]:
#I am now updating a copy

In [ ]:
# yes seems like it's working :-)
#  hopefully it loads back up...

In [ ]:
# a modified code from first_branch